In [17]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
%matplotlib inline
print("Version of tensorflow is : ",tf.__version__)

Version of tensorflow is :  2.4.0


In [2]:
BATCH_SIZE = 32 
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [3]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
def process_input(images):
    img = images.astype('float')
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img

In [5]:
X_train = process_input(training_images)
X_val   = process_input(validation_images)

In [6]:
print("Shape of X_train : ",X_train.shape)
print("Shape of training_labels : ",training_labels.shape)
print("Shape of X_val : ",X_val.shape)
print("Shape of validation_labels : ",validation_labels.shape)

Shape of X_train :  (50000, 32, 32, 3)
Shape of training_labels :  (50000, 1)
Shape of X_val :  (10000, 32, 32, 3)
Shape of validation_labels :  (10000, 1)


In [7]:
def predefine_model(inputs):
    resnet = tf.keras.applications.resnet.ResNet50(weights='imagenet', include_top=False , input_shape=(224,224,3))
    x = resnet(inputs)
    return x

In [13]:
def outputlayer(inputs):
    x = predefine_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024 , activation = 'relu')(x)
    x = tf.keras.layers.Dense(512  , activation = 'relu')(x)
    x = tf.keras.layers.Dense(10   , activation = 'softmax' , name="classification")(x)
    
    return x

In [14]:
def get_model():
    Input = tf.keras.layers.Input(shape=(32 , 32 , 3))
    x     = tf.keras.layers.UpSampling2D(size=(7,7))(Input)
    x     = outputlayer(x)

    model = tf.keras.models.Model(inputs = Input , outputs = x)
    model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy , optimizer = 'sgd' , metrics = ['acc'])
    return model

In [15]:
model = get_model()

In [16]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_3 (Dense)              (None, 512)               5248

In [19]:
epoch = 4
his = model.fit(X_train , training_labels ,
                epochs=epoch , batch_size=32 ,
                validation_data=(X_val , validation_labels) ,
                validation_batch_size=32)

Epoch 1/4
1563/1563 [==============================] - 559s 351ms/step - loss: 0.5975 - acc: 0.8054 - val_loss: 0.1781 - val_acc: 0.9419
Epoch 2/4
1563/1563 [==============================] - 553s 354ms/step - loss: 0.0955 - acc: 0.9697 - val_loss: 0.1687 - val_acc: 0.9471
Epoch 3/4
1563/1563 [==============================] - 553s 354ms/step - loss: 0.0406 - acc: 0.9869 - val_loss: 0.1639 - val_acc: 0.9514
Epoch 4/4
1563/1563 [==============================] - 555s 355ms/step - loss: 0.0228 - acc: 0.9930 - val_loss: 0.1579 - val_acc: 0.9543
